<h2>Based on assets and amenities data from Indian census prepare an index rankings the districts in the state of Maharashtra.</h2>
<ul>
    <li>
        First downloaded the dataset from the mentioned census website
    </li>
    <li>
        Since the column names were obscure changed them to more readable names
    </li>
    <li>
        Dataset has many variables which record the state of people having or not having basic amenities and assets, but I considered only percentage of people <b>having</b> certain amenities or assets to prepare this index.
    </li>
    <li>
        Lowest level of categorization was done at sub-district level, so had to take into account while preparing weight for a particular district.
    </li>
    <li>
        Earlier wanted to give separate weights to data provided for rural and urban regions but found out that some of the districts do not provide data for both rural and urban districts, so instead only considered <b>total</b> variable. 
    </li>
    <li>
        Add up all the variables for all the sub-districts for a district.
    </li>
    <li>
        Divide by number of sub-districts so that districts with many sub-districts do not dominate.
    </li>
    <li>
        Sort the series in a descending order to list various districts with value of weights associated with them.
    </li>
</ul>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline

In [7]:
census = pd.read_csv('./data/census_modified.csv', header=None); census.head()

,0,1,2,3,4,5,6,7,8,9,...,135,136,137,138,139,140,141,142,143,144
0,27,MAHARASHTRA,0,MAHARASHTRA,0,MAHARASHTRA,0,0,State - MAHARASHTRA,Total,...,24.9,5.9,7.2,19.0,68.4,26.3,4.3,2.6,1.7,1.0
1,27,MAHARASHTRA,0,MAHARASHTRA,0,MAHARASHTRA,0,0,State - MAHARASHTRA,Rural,...,18.5,2.4,1.3,29.8,53.4,38.9,6.8,4.1,2.6,1.0
2,27,MAHARASHTRA,0,MAHARASHTRA,0,MAHARASHTRA,0,0,State - MAHARASHTRA,Urban,...,32.6,10.1,14.4,6.0,86.5,11.1,1.3,0.7,0.6,1.0
3,27,MAHARASHTRA,497,Nandurbar,0,Nandurbar,0,0,District - Nandurbar,Total,...,12.3,1.9,1.4,54.8,27.3,60.1,11.9,7.5,4.4,0.7
4,27,MAHARASHTRA,497,Nandurbar,0,Nandurbar,0,0,District - Nandurbar,Rural,...,8.4,1.3,0.5,61.7,20.3,65.6,13.4,8.4,5.0,0.7


In [46]:
# lets take only the subset of features required by us
cols_to_use = [3, 9, 11, 12, 24, 25, 26, 27, 28, 29,
              34, 35, 36, 37, 38, 39, 42, 43, 44, 45, 
               46, 47, 50, 51, 52, 53, 54, 55, 56, 57,
              58, 62, 65, 66, 71, 72, 73, 75, 76, 84,
              86, 90, 112, 113, 114, 115, 126, 127, 128,
              129, 132, 134, 135, 136, 137, 139, 140]

In [47]:
census = pd.read_csv('./data/census_modified.csv',
                    header=None,
                    usecols=cols_to_use); census.head()

,3,9,11,12,24,25,26,27,28,29,...,127,128,129,132,134,135,136,137,139,140
0,MAHARASHTRA,Total,64.1,31.6,7.5,14.1,0.6,2.9,37.3,30.2,...,19.5,56.8,5.8,53.7,30.5,24.9,5.9,7.2,68.4,26.3
1,MAHARASHTRA,Rural,56.5,37.1,11.5,20.4,0.4,2.3,42.5,12.0,...,12.1,37.9,0.8,45.0,30.2,18.5,2.4,1.3,53.4,38.9
2,MAHARASHTRA,Urban,73.1,25.0,2.6,6.4,0.9,3.5,31.1,52.0,...,28.3,79.4,11.7,64.2,30.8,32.6,10.1,14.4,86.5,11.1
3,Nandurbar,Total,50.3,45.9,38.2,16.5,0.4,1.5,18.4,9.8,...,7.2,26.0,0.8,26.7,12.7,12.3,1.9,1.4,27.3,60.1
4,Nandurbar,Rural,47.2,48.9,43.9,18.1,0.2,1.3,15.2,4.5,...,7.2,18.7,0.4,20.7,10.2,8.4,1.3,0.5,20.3,65.6


In [48]:
census['per_homes_with_good_materials_for_roof'] = (census[24] + census[25] + census[26] + 
                                                    census[27] + census[28] + census[29])  

In [49]:
census['per_homes_with_good_materials_for_wall'] = (census[34] + census[35] + census[36] +
                                                    census[37] + census[38] + census[39])

In [50]:
census['per_homes_with_good_materials_for_floor'] = (census[42] + census[43] + census[44] +
                                                    census[45] + census[46])

In [51]:
census['per_families_with_more_than_2_rooms'] = (census[50] + census[51] + census[52]
                                                + census[53] + census[54])

In [52]:
census['per_families_with_household_less_than_4'] = (census[55] + census[56] + census[57]
                                                + census[58])

In [53]:
census['ownership_status'] = census[62]
census['per_married_couple'] = census[65] + census[66]
census['per_families_with_good_source_of_drinking_water'] = (census[71] + census[72] + 
                                                             census[73] + census[75] +
                                                             census[76]
                                                            )

In [54]:
census['per_families_with_good_source_of_lighting'] = census[84] + census[86]
census['per_families_with_proper_defecating_facility'] = census[90]
census['per_families_using_proper_fuel_for_cooking'] = census[112] + census[113] + census[114] + census[115]
census['per_families_availing_banking_facility'] = census[127]

In [55]:
census['per_family_with_tv_or_comp'] = census[137]
census['per_family_with_modes_of_transport'] = census[134] + census[135] + census[136]
census['per_family_with_good_quality_of_household'] = census[139] + census[140]

In [58]:
census_modified = census[census.columns[57:]]

In [144]:
census_modified.loc[:, ('district_name')] = census[3]
census_modified.loc[:, ('total_or_rural_or_urban')] = census[9]

In [145]:
census_modified.head()

,per_homes_with_good_materials_for_roof,per_homes_with_good_materials_for_wall,per_homes_with_good_materials_for_floor,per_families_with_more_than_2_rooms,per_families_with_household_less_than_4,ownership_status,per_married_couple,per_families_with_good_source_of_drinking_water,per_families_with_good_source_of_lighting,per_families_with_proper_defecating_facility,per_families_using_proper_fuel_for_cooking,per_families_availing_banking_facility,per_family_with_tv_or_comp,per_family_with_modes_of_transport,per_family_with_good_quality_of_household,district_name,total_or_rural_or_urban
0,92.6,72.3,62.8,53.7,53.7,81.1,82.4,85.7,84.1,53.1,50.7,19.5,7.2,61.3,94.7,MAHARASHTRA,Total
1,89.1,58.0,39.5,53.8,50.5,90.5,80.2,76.4,74.2,38.0,20.7,12.1,1.3,51.1,92.3,MAHARASHTRA,Rural
2,96.5,89.3,91.0,53.5,57.4,69.8,85.1,96.7,96.3,71.3,86.6,28.3,14.4,73.5,97.6,MAHARASHTRA,Urban
3,84.8,30.9,25.6,35.2,44.6,91.5,81.0,87.5,64.1,28.7,18.0,7.2,1.4,26.9,87.4,Nandurbar,Total
4,83.2,23.9,16.4,29.4,44.5,94.9,81.3,85.6,59.5,22.3,9.0,7.2,0.5,19.9,85.9,Nandurbar,Rural


In [146]:
census_modified_with_only_districts = census_modified.loc[3:]
census_modified_with_only_districts = census_modified_with_only_districts[census_modified_with_only_districts['total_or_rural_or_urban'] == 'Total']

In [147]:
census_modified_with_only_districts.head()

,per_homes_with_good_materials_for_roof,per_homes_with_good_materials_for_wall,per_homes_with_good_materials_for_floor,per_families_with_more_than_2_rooms,per_families_with_household_less_than_4,ownership_status,per_married_couple,per_families_with_good_source_of_drinking_water,per_families_with_good_source_of_lighting,per_families_with_proper_defecating_facility,per_families_using_proper_fuel_for_cooking,per_families_availing_banking_facility,per_family_with_tv_or_comp,per_family_with_modes_of_transport,per_family_with_good_quality_of_household,district_name,total_or_rural_or_urban
3,84.8,30.9,25.6,35.2,44.6,91.5,81.0,87.5,64.1,28.7,18.0,7.2,1.4,26.9,87.4,Nandurbar,Total
6,95.2,17.7,10.3,25.8,44.2,93.2,89.5,74.4,50.6,16.2,5.6,7.6,0.3,10.4,95.3,Nandurbar,Total
9,94.7,13.2,4.8,12.2,33.7,97.3,87.7,50.7,59.5,6.2,2.9,6.3,0.2,4.2,95.0,Nandurbar,Total
12,85.7,24.0,19.0,35.2,42.8,93.2,79.9,93.9,64.2,31.7,15.3,8.3,1.1,29.0,87.6,Nandurbar,Total
15,70.3,35.6,33.1,38.4,43.0,88.6,78.9,96.6,65.2,28.9,24.9,7.0,2.0,34.5,75.7,Nandurbar,Total


In [148]:
# lets print out all the districts
census_modified_with_only_districts.district_name.unique()

array(['Nandurbar', 'Dhule', 'Jalgaon', 'Buldana', 'Akola', 'Washim',
       'Amravati', 'Wardha', 'Nagpur', 'Bhandara', 'Gondiya', 'Gadchiroli',
       'Chandrapur', 'Yavatmal', 'Nanded', 'Hingoli', 'Parbhani', 'Jalna',
       'Aurangabad', 'Nashik', 'Thane', 'Mumbai Suburban', 'Mumbai',
       'Raigarh', 'Pune', 'Ahmadnagar', 'Bid', 'Latur', 'Osmanabad',
       'Solapur', 'Satara', 'Ratnagiri', 'Sindhudurg', 'Kolhapur', 'Sangli'], dtype=object)

In [149]:
# lets group this dataset by the district name
census_group_by = census_modified_with_only_districts.groupby('district_name')

In [162]:
count = 0
district_names = []
weights = []

for name, group in census_group_by:
    sum_group = 0
    
    for i in range(group.shape[0]):
        sum_total_sub_district = group.iloc[i:i+1,0:-2].sum(axis=1)
        sum_group = sum_group + (sum_total_sub_district.iloc[0])
            
    district_names.append(name)
    weights.append(sum_group / group.shape[0])

In [163]:
final_index = pd.DataFrame({'District': district_names, 'Weights': weights})

In [164]:
final_index = final_index.set_index('District')

In [165]:
final_index.Weights.order(ascending=False)

District
Mumbai Suburban    1082.800000
Mumbai             1073.900000
Kolhapur            999.546154
Pune                996.926667
Sangli              989.145455
Satara              985.216667
Nagpur              972.866667
Raigarh             961.118750
Wardha              927.800000
Bhandara            903.812500
Ratnagiri           896.150000
Sindhudurg          894.844444
Thane               887.693750
Chandrapur          886.631250
Solapur             875.600000
Ahmadnagar          867.333333
Gondiya             848.555556
Latur               844.436364
Amravati            839.820000
Osmanabad           837.877778
Akola               833.850000
Jalgaon             821.212500
Yavatmal            818.052941
Aurangabad          813.460000
Buldana             813.042857
Gadchiroli          805.869231
Washim              802.242857
Parbhani            801.990000
Nashik              799.900000
Hingoli             794.283333
Jalna               793.111111
Nanded              787.505882